# Análise do portal da transparencia da Câmara Lesgislativa de Caruaru para raspagem de dados referentes à Cota Parlamentar

data de inicio: 2023-09-01

data de conclusão: 2023-09-10

## Descrição

Em 17 de Fevereiro de 2022, a Câmara de Vereadores de Caruaru aprovou o Projeto de Lei Complementar nº 114/2022, que estabelece uma série de mudanças no orçamento interno da casa, entre elas, a criação de uma cota de alimentação e combustível para o exercício da atividade parlamentar dos 23 vereadores do município de até `R$` 2000,00 além de uma verba de até `R$` 9000,00 para despesas da presidência da câmara.  

O projeto foi apresentado pela Mesa Diretora sob a justificativa de ser necessária devido, principalmente, "aos impáctos orçamentários no duodécimo de 2021" que, segundo o presidente da câmara de vereadores, Bruno Lambreta (PSDB), [em entrevista à radio CBN de Caruaru](https://www.cbncaruaru.com/artigo/presidente-da-camara-de-vereadores-de-caruaru-responde-a-polemicas-envolvendo-projeto-de-cotas-parlamentares), "teria sofrido uma queda de 4.8%". Ainda de acordo com Lambreta, outra medida necessária seria o reajuste dos salários dos servidores que, segundo ele, não eram reajustados desde 2014.

Por fim, o projeto também permite que os vereadores e acessores reajustem valores de gratificações e diárias. O salário atual de um vereador é de R$ 15000,00

## Objetivos

- [x] Obter a url de todos os arquivos pdf que contém os dados do consumo da cota parlamentar vereadores do município de Caruaru.
- [ ] Baixar e extrair os dados do arquivo pdf para um formato tabular.


## Páginas

|NOME|URL|IMAGEM EXEMPLO|
|----|---|-----------|
|Página inicial|https://transparenciape.com.br/CamaraCaruaru/index.php|<img src="../img/Screenshot_2023-09-06-16-40-35_pagina_inicial.png"></img>|
|Cota para atividade parlamentar/Verba Idenizatória|https://transparenciape.com.br/CamaraCaruaru/cotaAtividadeParlamentar.php|<img src="../img/2023-09-01_18-32-cota-para-atividade-parlamentar.jpg"></img>|


## Mapeamento dos elementos de Interesse

### Tabela de Dados

As informações de interesse são acessadas através da tabela de dados. Quando a página é carregada, podemos navegar até o elemento `table` seguindo o caminho `//table[@id="tabelaDados`.

![elemento de interesse 1](../img/2023-09-03_20-02-elemento_de_interesse_1.png)

### Botões de Acesso ao documento

Os dados da tabela apresentada na página em si não são relevantes, assim como os arquivos gerados pelos botões "Gerar PDF," "Gerar XLS," e "Gerar CSV." Esses elementos produzem as mesmas informações apresentadas na tabela da página, ou seja, não fornecem novos dados, mas exatamente as informações presentes na tabela do site. Você pode verificar um exemplo desses arquivos no diretório example_files.

O que estamos buscando são as informações contidas nos arquivos PDF que podem ser obtidos ao clicar no botão "acessar arquivo" na coluna "AÇÕES" da tabela. Portanto, este é o segundo elemento de interesse. A URL do arquivo é passada como argumento para a função `window.open()` no atributo `onclick` do elemento `button`. O endpoint da URL é um documento PDF com o que parece ser um hash que identifica o arquivo. Ao clicar no botão, somos redirecionados para uma nova janela onde podemos baixar o PDF. Para localizar os 10 botões de cada página da tabela, por exemplo, o XPath é o seguinte: `//table[@id="tabelaDados"]//button[@class="btn btn-info btn-xs command-edit"]`:

![elemento 2](../img/2023-09-03_20-14-elemento_de_interesse_2.png)

### Obtendo o documento

Ao realizar a requisição para o arquivo pdf, clicando no botão "acessar arquivo", verificamos que a url é a mesma passada para `window.open()` : `http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/{id_arquivo}`

![Requisição](../img/2023-09-03_00-09-requisição.png)

Esta é a requisição chave para obtermos o arquivo de dados que desejamos. Contudo, clicar em cada botão é contra produtivo. Além disso, até o momento da redação desse estudo, quando selecionamos um mês qualquer na caixa "Filtros de Consulta", estranhamente somos enviados para outra página, "Folha de Pagamentos" que, no momento, não nos interessa. Portanto, é fundamental entendermos qual a melhor forma de obermos a url chave investigando mais a fundo como esse site funciona. 

### Código fonte da página

Minha primeira intuição para uma estratégia de raspagem, basicamente, seria raspar todos os elementos `button` da tabela, depois extrair a url referente ao documento da função `window.opent()` com uma expressão regular com o intuito de, por fim, utilizar essa url limpa para realizar as requisições de cada documento. Porém, ao desabilitar o JavaScript* do navegador, percebemos que a tabela não é renderizada, ou seja, é um recurso interativo dependente de um script que busca os dados no backend para montá-la. 

`Ctrl+Shift+P` para abrir a paleta de comandos do inspetor. Procure por "Desativar JavaScript": 
![dasativar_javascript](../img/2023-09-10_18-49-destivar_javascript.png)

Ao atualizarmos a página, vemos que a tabela não é carregada: 
![desabilita_js](../img/2023-09-08_00-16-desabilita_js.png)



Uma outra forma de checar isso sem desativar o JavaScript é analisando o código fonte da página. Buscando pelo ID do elemento `table`, "tabelaDados", encontramos a estrutura HTML que apenas renderiza o cabeçalho da tabela:

![font](../img/2023-09-03_18-32-codigo_fonte.png)

Scripts, geralmente, serão executados quando a página inteira estiver rederizada, por isso é comum que sejam as últimas tags em uma estrutura HTML. Neste caso não é diferente. As próximas imagens mostram o script que configura a tabela interativa que nos levará até o documento que buscamos. 

Podemos dividir esse script em 4 partes: 

1. O código está encapsulado em uma função anônima que é executada quando o documento HTML é carregado completamente (quando o DOM está pronto). Isso é feito usando `$(function() {...});`, que é uma abreviação do jQuery para o evento `$(document).ready()`. 

Ele configura uma tabela no elemento `table` com ID "tabelaDados" usando o plugin bootgrid. Bootgrid é um plugin jQuery para criar tabelas interativas com recursos de pesquisa, ordenação e paginação.

A tabela bootgrid é configurada com várias opções, como a **configuração de [AJAX](https://developer.mozilla.org/pt-BR/docs/Web/Guide/AJAX)** para buscar dados do servidor sem a necessidade de recarregar a página inteira, permitindo a seleção de linhas, definindo uma função para enviar dados extras para o servidor, **especificando a URL de onde os dados serão buscados**, **formatando as células da coluna "commands" para criar um botão que abre um arquivo em uma nova janela** e definindo uma mensagem de "nenhum resultado" personalizada.

2. Define o atributo "placeholder" do elemento com o ID "search-field" para "Procure por data ou descrição." Isso é usado para fornecer uma dica de pesquisa na caixa de entrada. Também configura elementos de entrada de data usando o plugin "datepicker" do jQuery para fornecer uma interface de seleção de data amigável ao usuário.


![script1](../img/2023-09-03_19-37-script_explain.png)

3. Fora da função anônima, há algumas funções adicionais:

    3.1 `gerarData(str)`: Esta função divide uma string de data no formato "dd/mm/yyyy" em partes e cria um objeto de data JavaScript correspondente.

    3.2 `verificar()`: Esta função verifica se a data final é maior do que a data inicial e exibe uma mensagem de erro se não for.

4. `gerarPDF()`, `gerarXLS()`, `gerarCSV()`: Essas funções são chamadas quando os usuários clicam em botões para gerar arquivos PDF, XLS (Excel) ou CSV. Elas constroem URLs com parâmetros de pesquisa e abrem esses URLs em uma nova janela.

![script1](../img/2023-09-03_19-37-script_explain1.png)

De forma resumida, assim que a requisição é feita para a url `https://transparenciape.com.br/CamaraCaruaru/cotaAtividadeParlamentar.php` uma segunda requisição é realizada à um banco de dados para obter as informações necessárias para preencher a tabela. Esta segunda requisição é feita de forma assíncrona graças ao modelo AJAX implementado, ou seja, recebemos sua resposta independentemente da resposta da requisição ao servidor onde a página está hospedada (supondo que esteja em um servidor diferente). Isso faz sentido porque o script precisa das informações acessíveis para montar a tabela. Note que, para montar a url fornecida para função `window.open()`, o script concatena parte de uma url com uma certa propriedade `row.arquivo`, indicando que a resposta da requisição assíncrona possui um campo chamado `arquivo` cujo valor, provavelmente, é aquela sequencia numérica de extenção `.pdf` que vemos no final da url. 

Portanto, o próximo passo é checar as requisições e encontrar a fonte dos dados utilizados pelo script para preencer a tabela.

## Analisando as Requisições

Ao analisar as requisições feitas, encontramos o arquivo `xhr` que  é a resposta da requisição assíncrona (AJAX) realizada no momento da requisição inicial para fornecer as informações que serão usadas pra montar a tabela:

![json](../img/2023-09-04_22-59-requisição_documentos.png)

### XMLHttpRequest
O arquivo xhr refere-se a um objeto [`XMLHttpRequest`](https://developer.mozilla.org/pt-BR/docs/Web/API/XMLHTTPRequest), que oferece a facilidade de transmitir dados entre o cliente e o servidor sem a necessidade de atualizar a página inteira. Em outras palavras, ele permite que uma página da web atualize apenas parte de seu conteúdo sem interromper o que o usuário está fazendo. O objeto XMLHttpRequest envia solicitações a um servidor e recebe respostas em formatos como texto, XML, JSON ou outros, dependendo do tipo de dados solicitados. É também amplamente utilizado na programação AJAX.

Se observarmos a resposta, veremos que, embora o Content-Type indique que recebemos um documento text/html, na verdade, podemos verificar que o formato da resposta é JSON:

![resposta](../img/2023-09-10_22-34-cabeçalho_resposta.png)

![resposta_json](../img/2023-09-05_01-26-resposta_json.png)

É provável que lógica da API que está gerando a resposta possa estar configurada de forma inadequada e enviando os dados no formato JSON, mas não definindo o cabeçalho Content-Type corretamente. Contudo, isso não impede de avançarmos.

### Acesso direto aos dados

Agora, temos acesso à url da requisição `xhr`:

![url_assync](../img/2023-09-04_23-05-querie_string_xhr.png)

Tudo indica que, com essa informação em mãos, podemos requisitar os dados diretamente ao servidor sem necessáriamente interagir com a página em si. Note que é enviada uma série de parâmetros de consulta ao servidor. Decompondo a url, temos:

- `https://transparenciape.com.br`: Este é o esquema e o domínio da URL. "https://" indica que está sendo usado o protocolo seguro HTTPS para a comunicação, e "transparenciape.com.br" é o nome de domínio do site ao qual se está acessando.

- `/CamaraCaruaru/cotaAtividadeParlamentarClass.php`: Esta parte da URL indica o caminho no servidor ao qual estamos acessando. .

    - `?dataInicial=01/01/2000&dataFinal=06/09/2023&current=1&rowCount=10&sort%5Bdata_publicacao_f%5D=desc&searchPhrase=&id=b0df282a-0d67-40e5-8558-c9e93b7befed`: Esta é a parte da URL que contém os parâmetros da consulta ou solicitação que são enviados ao servidor para que ele retorne informações específicas. Aqui estão vários parâmetros separados por "&". Os parâmetros incluem:
    
    - `dataInicial=01/01/2000`: Isso indica uma data de início, acredito que 1 de janeiro de 2000 seja uma data definida arbitrariamente, pois não existem dados dessa data.
    -  `dataFinal=06/09/2023`: Isso indica uma data de término. Por padrão essa é a mesma data da requisição (no caso, minha requisição foi 06/09).
    - `current=1`: Parece ser um parâmetro que indica a página atual ou o número da página na consulta. Neste caso, está definido como 1.
    - `rowCount=10`: Indica a quantidade de linhas ou elementos que devem ser exibidos em cada página. Aqui, está definido como 10.
    - `sort%5Bdata_publicacao_f%5D=desc`: Parece ser um parâmetro de ordenação que indica como os resultados devem ser ordenados. Neste caso, parece estar ordenando pela data de publicação em ordem decrescente (do mais novo para o mais antigo).
    - `searchPhrase=`: Este parâmetro parece estar vazio, o que pode indicar que não está sendo feita uma pesquisa específica nesta solicitação.
    - `id=b0df282a-0d67-40e5-8558-c9e93b7befed`: Este parâmetro parece ser um identificador único ou código que é usado na solicitação, mas o seu propósito específico não está claro sem mais contexto... isso, talvez a gente nunca vá saber.

Para testar essa hipótese, vamos fazer uma solicitação usando a mesma url:

In [1]:
import requests

In [2]:
url = "https://transparenciape.com.br/CamaraCaruaru/cotaAtividadeParlamentarClass.php?dataInicial=01/01/2000&dataFinal=04/09/2023&current=1&rowCount=10&sort%5Bdata_publicacao_f%5D=desc&searchPhrase=&id=b0df282a-0d67-40e5-8558-c9e93b7befed"


In [3]:
response = requests.get(url)

In [4]:
response

<Response [200]>

In [5]:
data = response.json()

In [6]:
data

{'current': 1,
 'rowCount': 10,
 'total': 32,
 'rows': [{'usuario': '6',
   'id': '34',
   'data_publicacao': '2023-09-04',
   'descricao': 'Cota Parlamentar Alimentação - Agosto 2023',
   'arquivo': '1693859460_f332d580a892ce100b3f.pdf',
   'created_at': '2023-09-04 17:31:00',
   'updated_at': '2023-09-04 17:31:00',
   'data_publicacao_f': '04/09/2023'},
  {'usuario': '6',
   'id': '33',
   'data_publicacao': '2023-09-04',
   'descricao': 'Cota Parlamentar Combustível - Agosto 2023',
   'arquivo': '1693859429_8799463dbd41a51af7a6.pdf',
   'created_at': '2023-09-04 17:30:29',
   'updated_at': '2023-09-04 17:30:29',
   'data_publicacao_f': '04/09/2023'},
  {'usuario': '6',
   'id': '31',
   'data_publicacao': '2023-07-31',
   'descricao': 'Cota Parlamentar Alimentação - Julho 2023',
   'arquivo': '1691176118_1519d141bf345198914b.pdf',
   'created_at': '2023-08-04 16:08:39',
   'updated_at': '2023-08-04 16:08:39',
   'data_publicacao_f': '31/07/2023'},
  {'usuario': '6',
   'id': '30',
 

Como esperávamos, a resposta dessa requisição são os dados reais usados para preencher a tabela interativa do site. Isso permite que possamos buscar exatamente os dados que precisamos, neste caso, `data_publicacao`, `descricao` e, o mais importante, `arquivo` cujo valor é o endpoint da url  passada para o evento `onclick` do botão "acessar arquivo". 

### Obtendo todos os resultados disponíveis

Embora tenhamos conseguido acessar os dados diretamente, ainda precisamos observar um detalhe: A tabela exibida no site é carregada com 10 resultados por padrão:
![opções_da_tabela](../img/2023-09-05_00-14_barra_de_opções_da_tabela.png)

Esse valor é passado na query string da requisição: 
![queriestring_padrão](../img/2023-09-05_00-22-queriestring_padrão.png)

É isso que recebemos como resposta nos campos `total` e `rowCount`. O campo `current` indica que estamos na "página 1". Nosso objetivo é obter todos os resultados disponíveis. Para descobrir qual valor atribuir à variável `rowCount` para obter do servidor todos os resultados, foi necessário apenas selecionar a opção "Todos" no menu da tabela:
![todos_os_resultados](../img/Screenshot_2023-09-05_00-15-22-seleciona_todos.png)

E checar qual valor é atribuido à `rowCount` na query string da requisição assíncrona:
![queriestring_todos](../img/2023-09-05_00-19_queriestring_para_todos_os_resultados.png)

Como podemos notar, -1 é passado para `rowCount` para retorna todos os resultados. Vamos testar uma requisição com esse valor:

In [7]:
response = requests.get(
    'https://transparenciape.com.br/CamaraCaruaru/cotaAtividadeParlamentarClass.php?dataInicial=01/01/2000&dataFinal=04/09/2023&current=1&rowCount=-1&sort%5Bdata_publicacao_f%5D=desc&searchPhrase=&id=b0df282a-0d67-40e5-8558-c9e93b7befed',
)

In [15]:
response

<Response [200]>

In [9]:
response.json()

{'current': 1,
 'rowCount': 10,
 'total': 32,
 'rows': [{'usuario': '6',
   'id': '34',
   'data_publicacao': '2023-09-04',
   'descricao': 'Cota Parlamentar Alimentação - Agosto 2023',
   'arquivo': '1693859460_f332d580a892ce100b3f.pdf',
   'created_at': '2023-09-04 17:31:00',
   'updated_at': '2023-09-04 17:31:00',
   'data_publicacao_f': '04/09/2023'},
  {'usuario': '6',
   'id': '33',
   'data_publicacao': '2023-09-04',
   'descricao': 'Cota Parlamentar Combustível - Agosto 2023',
   'arquivo': '1693859429_8799463dbd41a51af7a6.pdf',
   'created_at': '2023-09-04 17:30:29',
   'updated_at': '2023-09-04 17:30:29',
   'data_publicacao_f': '04/09/2023'},
  {'usuario': '6',
   'id': '31',
   'data_publicacao': '2023-07-31',
   'descricao': 'Cota Parlamentar Alimentação - Julho 2023',
   'arquivo': '1691176118_1519d141bf345198914b.pdf',
   'created_at': '2023-08-04 16:08:39',
   'updated_at': '2023-08-04 16:08:39',
   'data_publicacao_f': '31/07/2023'},
  {'usuario': '6',
   'id': '30',
 

Observe apenas que, apesar de passarmos -1 para `rowCount` na query string, o valor na resposta continua sendo 10... Para checar o total de resultados obtidos bate com o total informado pelo site, podemos contar os itens do campo `rows` da resposta:

In [10]:
len(response.json()["rows"])

32

### Encurtando a url de acesso

Analisando a url que dá acesso aos dados notamos que os valores inprecindíveis são `dataInicial`, `dataFinal` e `rowCount`. A ausencia das outras variáveis não impedem de termos o retorno desejado:

In [2]:
response = requests.get(
    'https://transparenciape.com.br/CamaraCaruaru/cotaAtividadeParlamentarClass.php?dataInicial=01/01/2000&dataFinal=04/09/2023&rowCount=-1',
)

In [3]:
responsoe.json()

{'current': 0,
 'rowCount': 10,
 'total': 32,
 'rows': [{'usuario': '6',
   'id': '34',
   'data_publicacao': '2023-09-04',
   'descricao': 'Cota Parlamentar Alimentação - Agosto 2023',
   'arquivo': '1693859460_f332d580a892ce100b3f.pdf',
   'created_at': '2023-09-04 17:31:00',
   'updated_at': '2023-09-04 17:31:00',
   'data_publicacao_f': '04/09/2023'},
  {'usuario': '6',
   'id': '33',
   'data_publicacao': '2023-09-04',
   'descricao': 'Cota Parlamentar Combustível - Agosto 2023',
   'arquivo': '1693859429_8799463dbd41a51af7a6.pdf',
   'created_at': '2023-09-04 17:30:29',
   'updated_at': '2023-09-04 17:30:29',
   'data_publicacao_f': '04/09/2023'},
  {'usuario': '6',
   'id': '31',
   'data_publicacao': '2023-07-31',
   'descricao': 'Cota Parlamentar Alimentação - Julho 2023',
   'arquivo': '1691176118_1519d141bf345198914b.pdf',
   'created_at': '2023-08-04 16:08:39',
   'updated_at': '2023-08-04 16:08:39',
   'data_publicacao_f': '31/07/2023'},
  {'usuario': '6',
   'id': '30',
 

Podemos, portanto usar essa url um pouco mais enxuta para as requisições.

Por fim, para obter todas as urls dos documentos, basta concatenar as primeiras partes da url do documento, com o valor do campo "arquivo" da resposta:

In [11]:
urls_doc = []

base_url = "http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/"

for item in response.json()["rows"]:
    urls_doc.append(base_url + item["arquivo"])

print(urls_doc)

['http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/1693859460_f332d580a892ce100b3f.pdf', 'http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/1693859429_8799463dbd41a51af7a6.pdf', 'http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/1691176118_1519d141bf345198914b.pdf', 'http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/1691176097_2c94c4fc4496a075346b.pdf', 'http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/1690204413_f7193145a91706d0058f.pdf', 'http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/1690204278_f5069ee1a5f27921c41a.pdf', 'http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/1690204389_41548f42698bb16144f9.pdf', 'http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/1690204258_3a8a53cbd088da5e8c0b.pdf', 'http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/1690204369_7487a61ace30ab32861b.pdf', 'http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/1690204233_97895077913842f1d308.pdf',

In [12]:
len(urls_doc)


32

## Resumo

### Elementos de interesse

Os dados que queremos obter estão contidos nos seguintes elementos:

|NOME|PRESENTE NA PÁGINA|SELETOR|RELATIVO A|TIPO DE OBJETO|
|----|------------------|-------|----------|--------------|
|Botão de Acesso ao Documento que contém o link para o arquivo|Cota Para Atividade Parlamentar/Verba Idenizatoria|//table[@id="tabelaDados"]//button[@class="btn btn-info btn-xs command-edit"]/@onclick|Raiz|Elemento|
|Data do Documento|Cota Para Atividade Parlamentar/Verba Idenizatoria|./tr[@data-row-id]/td[1]/text()|Tabela|Dado|
|Descrição do Documento|Cota Para Atividade Parlamentar/Verba Idenizatoria|./tr[@data-row-id]/td[2]/text()|Tabela|Dado|

### Requisições

Contudo, identificamos a possibilidade de acessarmos os dados diretamente através de uma única requisição ao servidor. Esta requisição é feita por uma url contendo uma query string que retorna os dados do servidor em formato JSON. A resposta é usada para gerar a tabela de dados apresentada na página e inclui, entre outros dados, a data do documento, a descrição e o nome do arquivo. Dessa forma, podemos acessar os dados sem interagir com a página em si, armazenar as informações e, o mais importante, montar a url que ultilizaremos para baixar o arquivo pdf e extrair seus dados na próxima etapa.

Portanto, as requisições que faremos são as seguintes:

|NOME|DE|PARA|TIPO DA REQUISIÇÃO|URL|FORMDATA|
|----|--|----|------------------|---|--------|
|Requisição inicial api|N/A|Acesso à api|GET|https://transparenciape.com.br/CamaraCaruaru/cotaAtividadeParlamentarClass.php?dataInicial=01/01/2000&dataFinal={end_date}&rowCount=-1|N/A|
|Requisição ao documento pdf|Página para tabela de dados|Documento PDF|GET|http://transparencia.caruaru.pe.leg.br/sistema/uploads/cotas/{arquivo.pdf}|N/A|

